ETL (Extract Transform Load) Script for getting the data. We first scrape the data using bs4. See `webscraper.py`. Afterwards we get the election data and process them and merge them together into a single csv. 

**TODO how did we get the gemeinden list?**

We first run the scraping script. Since the script takes a long time (if we assume an HTTP request takes 0.5 seconds and parsing/extracting takes 0.1 seconds, the scraping would take almost an hour), we do not run the script right now for convenience. However, the data from a scrape (date accessed: 2/4/2025) is saved in `data/Bildungsausgaben_Gemeinden_Oberösterreich_data_2007_bis_2019.csv`

In [1]:
# %run webscraper.ipynb

Afterwards, we get the data for the elections. We will use spark from now on, to ensure scalability.

**TODO how did we get election data**

Since the election data was originally provided in an inconsistent XLS format—with irregular merged cells, random empty lines, and other structural issues—we manually formatted the data to ensure consistency and saved it as an XLSX file for better usability.

Since we had only 5 XLS files for election data, this was the easiest option. However, this can be automated using a script for formatting if we were to automate and scale this project. However, this is out of scope for now.

Afterward, we used a small script to convert these XLSX files to CSV. Given the limited number of election datasets and the straightforward nature of the conversion, the added complexity and overhead of Spark are not justified. Therefore, we will use Pandas for this conversion. However, if scalability in this specific conversion becomes a concern, the script can be rewritten to use PySpark.

In [2]:
import os
import pandas as pd

data_dir = os.path.join(os.getcwd(), "data")

# Get all .xlsx files in the data directory
xlsx_files = [f for f in os.listdir(data_dir) if f.endswith(".xlsx")]

for file in xlsx_files:
    file_path = os.path.join(data_dir, file)
    try:
        df = pd.read_excel(file_path, sheet_name=None)
        for sheet_name, sheet_df in df.items():
            # Replace spaces with underscores in filename
            sanitized_filename = os.path.splitext(file)[0].replace(" ", "_")
            sanitized_sheet_name = sheet_name.replace(" ", "_")
            csv_filename = os.path.join(data_dir, f"{sanitized_filename}_{sanitized_sheet_name}.csv")

            sheet_df.to_csv(csv_filename, index=False, sep=",", encoding="latin1")
            print(f"Converted: {file} (Sheet: {sheet_name}) -> {csv_filename}")
    except Exception as e:
        print(f"Error converting {file}: {e}")


Converted: OÖ 2017.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2017_Stimmen.csv
Converted: OÖ 2017.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2017_Mandate.csv
Converted: OÖ 2019.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2019_Stimmen.csv
Converted: OÖ 2019.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2019_Mandate.csv
Converted: OÖ 2024.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2024_Stimmen.csv
Converted: OÖ 2024.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2024_Mandate.csv
Converted: OÖ 2008.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2008_Stimmen.csv
Converted: OÖ 2008.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2008_Mandate.csv
Converted: OÖ 2013.xlsx (Sheet: Stimmen) -> /home/jovyan/DP2/Project/data/OÖ_2013_Stimmen.csv
Converted: OÖ 2013.xlsx (Sheet: Mandate) -> /home/jovyan/DP2/Project/data/OÖ_2013_Mandate.csv
Converted: OÖ 2013.xlsx (Sheet: whlsortgemnr) -> /home/jovya

OÖ 2013.xlsx contained 3 (hidden) sheets, which upon manual inspection, are not relevant and/or redundant for us. Therefore we remove them.

In [3]:
!rm data/OÖ_2013_whlsortgemnr.csv
!rm data/OÖ_2013_MandateData.csv
!rm data/OÖ_2013_WAHLDAT.csv

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, when, lit, expr
import pyspark.sql.functions as F
import os
from unidecode import unidecode # https://pypi.org/project/Unidecode/
# init spark
spark = (
    SparkSession.builder
    .appName("MunicipalSpendingAndElectionAnalysis")
    .getOrCreate()
)

Then we import the data from the CSV files and clean them. We start with the spendings data:
- we remove encoding issues we noticed upon manual inspection
- we rename categories for readability and cast their type
- we aggregate the spending data.
    - we do this becuase the data is currently broken down into subcategories (e.g. spendings on preschool education, on physical education etc.)
    - for some of the analysis we will use the total spending per municipality
- we slugify using the helper function the municipality names so we can process them easier

In [5]:
# helper function
def standardize_municipality(col_name):
    """lowercase, trim, replace spaces with dashes (i.e. slugify)"""
    return F.regexp_replace(F.lower(F.trim(F.col(col_name))), " ", "-")


# --- read & transform municipal spending data ---
spending_csv = "data/Bildungsausgaben_Gemeinden_Oberösterreich_data_2007_bis_2019.csv"
df_spending = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("encoding", "UTF-8")
    .csv(spending_csv)
)

# rename cols
df_spending = (df_spending
    .withColumnRenamed("Gemeinde", "Municipality")
    .withColumnRenamed("Year", "Year")
    .withColumnRenamed("Abschnitt", "Category")
    .withColumnRenamed("Betrag in Euro", "Spending")
)

# replace chars in category
df_spending = df_spending.withColumn(
    "Category",
    regexp_replace(col("Category"), "FĂ¶rderung", "Förderung")
)
df_spending = df_spending.withColumn(
    "Category",
    regexp_replace(col("Category"), "Sport und auĂźerschulische Leibeserziehung",
                   "Sport und außerschulische Leibeserziehung")
)

# cast cols
df_spending = df_spending.withColumn("Year", col("Year").cast("int"))
df_spending = df_spending.withColumn("Spending", col("Spending").cast("float"))

# aggregate spending
df_spending_agg = (
    df_spending
    .groupBy("Municipality", "Year")
    .agg(F.sum("Spending").alias("Spending_Summe"))
)

# add lowercase municipality
df_spending_agg = df_spending_agg.withColumn("Municipality_Lowercase", standardize_municipality("Municipality"))
print("Done")

Done


After this, we process the election data for the available years. We initialize an empty DataFrame with predefined columns to store the cleaned data. The predefined columns are consistent amongst Austrian election datasets. Therefore, this script would presumably work for any election data across Austria. For including other countries, at first, we would need to standardize their election data to this format.

We read the CSV into Spark, standardize column names, and identify party vote columns. We convert party vote counts to integers and determine the winning party for each municipality, which we will use for predictive analysis later on. We also add a slugified version of the municipality name for consistency.

Once cleaned, the election data is merged into a single DataFrame and joined with the spending data using slugified municipality name and year. Since spending data only covers 2007–2019, elections from 2024 are automatically excluded. However, upon adding recent spending data, 2024 would be automatically included without needing manual modification to this script. Finally, we save the merged dataset as a CSV.

In [6]:

# --- read & transform election data ---
election_yrs = [2008, 2013, 2017, 2019, 2024]
all_election_df = spark.createDataFrame([], schema="Year INT, Municipality_ID STRING, Municipality_Name STRING, Wahlberechtigte INT, abgegebene_Stimmen INT, gueltige_Stimmen INT, Wahlbeteiligung STRING, Winning_Party STRING, Municipality_Lowercase STRING")

for year in election_yrs:
    print(f"Processing data for {year}")
    votes_file = f"data/OÖ_{year}_Stimmen.csv"

    # skip if file missing
    if not os.path.exists(votes_file):
        print(f"[WARNING] Missing CSV(s) for year {year} -> Skipping.")
        continue

    # read csv
    votes_df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("encoding", "latin1")
        .csv(votes_file)
    )

    # normalize col names
    renamed_columns = {c: unidecode(c.strip()).replace(" ", "_") for c in votes_df.columns}
    votes_df = votes_df.selectExpr(*[f"`{old}` as `{new}`" for old, new in renamed_columns.items()])
    
    # identify party cols
    party_columns = [c for c in votes_df.columns if c not in {
        "Nr.", "Name", "Wahlberechtigte", "abgegeb._Stimmen", "gultige", "Wahlbet.", "ungultige"
    }]

    # rename cols
    votes_df = votes_df.withColumnRenamed("Nr.", "Municipality_ID")
    votes_df = votes_df.withColumnRenamed("Name", "Municipality_Name")
    votes_df = votes_df.withColumnRenamed("Wahlberechtigte", "Wahlberechtigte")
    votes_df = votes_df.withColumnRenamed("abgegeb._Stimmen", "abgegebene_Stimmen")
    votes_df = votes_df.withColumnRenamed("gultige", "gueltige_Stimmen")
    votes_df = votes_df.withColumnRenamed("Wahlbet.", "Wahlbeteiligung")

    # cast party cols to int
    for party_col in party_columns:
        votes_df = votes_df.withColumn(f"votes_{party_col}", col(party_col).cast("int"))

    # determine win party
    vote_cols = [F.coalesce(col(f"votes_{party_col}"), lit(0)) for party_col in party_columns]
    votes_df = votes_df.withColumn("max_votes", F.greatest(*vote_cols))

    winning_party_cases = [when(col("max_votes") == col(f"votes_{party_col}"), lit(party_col)) for party_col in party_columns]
    votes_df = votes_df.withColumn("Winning_Party", F.coalesce(*winning_party_cases))

    # drop temp cols
    for party_col in party_columns:
        votes_df = votes_df.drop(f"votes_{party_col}")

    votes_df = votes_df.drop("max_votes")

    # add year & municipality lowercase
    votes_df = votes_df.withColumn("Year", lit(year))
    votes_df = votes_df.withColumn("Municipality_Lowercase", standardize_municipality("Municipality_Name"))
    # filter for numeric municipality ids
    votes_df = votes_df.filter(col("Municipality_ID").rlike("^[0-9]+$"))

    # add to main df
    all_election_df = all_election_df.unionByName(votes_df.select(
        "Year", "Municipality_ID", "Municipality_Name", "Wahlberechtigte",
        "abgegebene_Stimmen", "gueltige_Stimmen", "Wahlbeteiligung", "Winning_Party", "Municipality_Lowercase"
    ))

# --- merging and output ---
# join dfs
merged_df = all_election_df.join(df_spending_agg, on=["Municipality_Lowercase", "Year"], how="inner").drop("Municipality")
# write to csv
merged_df.coalesce(1).write.mode("overwrite").option("header", "true").option("encoding", "latin1").csv("data/merged_data.csv")
print("Merged data sample:")
merged_df.show(5, truncate=False)

Processing data for 2008
Processing data for 2013
Processing data for 2017
Processing data for 2019
Processing data for 2024
Merged data sample:
+----------------------+----+---------------+------------------------------+---------------+------------------+----------------+---------------+-------------+-------------------+
|Municipality_Lowercase|Year|Municipality_ID|Municipality_Name             |Wahlberechtigte|abgegebene_Stimmen|gueltige_Stimmen|Wahlbeteiligung|Winning_Party|Spending_Summe     |
+----------------------+----+---------------+------------------------------+---------------+------------------+----------------+---------------+-------------+-------------------+
|linz                  |2008|40101          |Linz                          |142125         |96209             |94496           |67.69          |SPO          |1.203534975E8      |
|steyr                 |2008|40201          |Steyr                         |28962          |20765             |20335           |71.7       

In [ ]:
spark.stop()